In [1]:
import csv
import datetime
import json
import os
import pandas as pd
import re

Setting variables

In [2]:
path_wd   = os.getcwd() 
path_root = path_wd.rsplit("\\",1)[0]

path_data = path_root + '\\data\\'
path_etl  = path_root + '\\etl\\'

path_user = os.path.expanduser('~')

Data Extraction and Cleansing

In [3]:
# This function returns a data frame with ["full_path", "item", "path", "file", ("extension")] 
# root_path -> file path in windows format
# extension = [1, 0] -> extension of the file based on the name
def list_files(root_path, extension):
    if extension == 1:
        cols = ["full_path", "item", "path", "file", "extension"]
    else:
        cols = ["full_path", "item", "path", "file"]
    
    df_final = pd.DataFrame(columns = cols)
    df = pd.DataFrame(columns = cols)

    # List with all items into 'root_path'
    list_root_path = os.listdir(root_path)
    
    # Iterates through root_path items list and concat info: [cols] to final df
    for i in range(len(list_root_path)):
        item = list_root_path[i]
        full_path = root_path + item

        if os.path.isdir(full_path):
            path_list = os.listdir(full_path)

            # Iterates through directory items list 
            for j in range(len(path_list)):
                file = path_list[j]
                f_type = file.rsplit(".", 1)[1] # maxsplit parameter = 1 returns 2 values, then select list position value, same logic

                if extension == 1:
                    data = [full_path+'\\'+file, item, full_path, file, f_type]
                else:
                    data = [full_path+'\\'+file, item, full_path, file]

                df.loc[j] = data

        elif os.path.isfile(full_path):
            file = item
            f_type = file.rsplit(".", 1)[1]

            if extension == 1:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-2], root_path, file, f_type]], columns = cols)
            else:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-2], root_path, file]], columns = cols)

        else:
            file = item
            f_type = file.rsplit(".", 1)[1]

            if extension == 1:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-1], root_path, file, f_type]], columns = cols)
            else:
                df = pd.DataFrame([[full_path, full_path.rsplit("\\")[-1], root_path, file]], columns = cols)

        df_final = pd.concat([df_final, df], ignore_index = True)
        df = pd.DataFrame(columns = cols)
    
    return df_final

In [4]:
# This function clean csv file separators
def clean_csv_separator(input_file):
    try:
        output_file = path_etl + "cleansing\\" + input_file.rsplit("\\")[-1].replace('.csv', '_formatted.csv')

        with open(input_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
            rows = f_in.readlines()
            format_rows = []
            separators = [';;', '||', '|', '%', ';']

            for r in rows:
                for s in separators:
                    r = r.replace(s, ',')
                format_rows.append(r)

            f_out.writelines(format_rows)

        print(f"New file created: {output_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [5]:
# List all files in data directory
df_raw_files_list = list_files(path_data, 1)

CSV files

In [6]:
df_csv_list = df_raw_files_list[df_raw_files_list['extension'] == 'csv']

# iterate through the list and clean files
for i in range(len(df_csv_list)):
    f_name = df_csv_list['full_path'].iloc[i]

    df_clean = clean_csv_separator(f_name)

New file created: c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\BaseCargos_formatted.csv
New file created: c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\BaseCEP_formatted.csv
New file created: c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\BaseClientes_formatted.csv
New file created: c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\BaseFuncionarios_formatted.csv
New file created: c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\BaseNivel_formatted.csv
New file created: c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\BasePQ_formatted.csv


JSON files

In [7]:
# This function clean json file bugs
def clean_json_file(input_file):
    try:
        output_file = path_etl + "cleansing\\" + input_file.rsplit("\\")[-1].replace('.json', '_formatted.json')

        with open(input_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
            rows = f_in.readlines()
            format_rows = []

            separators = ["\";"]
            start_lines = ['\"{']
            end_lines = ['}\"']
            q_marks = ['"""', '""', '\'']

            for r in rows:
                for s in separators:
                    r = r.replace(s, ',')
                for t in start_lines:
                    r = r.replace(t, '{')
                for e in end_lines:
                    r = r.replace(e, '},')
                for q in q_marks:
                    r = r.replace(q, '"')
                format_rows.append(r)
            
            format_rows[0] = "[\n"
            format_rows[-1] = format_rows[-1].rstrip() + ']\n'
            format_rows[-1] = format_rows[-1].replace('},]', '}\n]')

            formatted_json = json.dumps(json.loads(''.join(format_rows)), indent=4)
            f_out.write(formatted_json)

        print(f"New file created: {output_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [8]:
df_json_list = df_raw_files_list[df_raw_files_list['extension'] == 'json']

# iterate through the list and clean files
for i in range(len(df_json_list)):
    f_name = df_json_list['full_path'].iloc[i]

    df_clean = clean_json_file(f_name)

print(df_clean)

New file created: c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\Vendas_formatted.json
None


In [9]:
def validate_json_file(f_name):
    try:
        with open(f_name, 'r') as f:
            json.load(f)
        print("File is valid.")

    except json.JSONDecodeError as e:
        print("File is invalid. Error message:")
        print(e)

f_name = r'c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\Vendas_formatted.json'
validate_json_file(f_name)

File is valid.


In [10]:
# List all files in etl directory
df_etl_files_list = list_files(path_etl, 1)

out_csv = path_etl+'list_etl_files.csv'
df_etl_files_list.to_csv(out_csv, index=False)
print(out_csv)

c:\Users\Diane\git\data_analysis\Nestle\etl\list_etl_files.csv


In [11]:
def export_json_to_csv(json_file, out_csv):
    df = pd.json_normalize(json_file, sep='.')
    df.to_csv(out_csv, index=False)
    print(f'JSON data was exported to "{out_csv}"')

In [12]:
df_json_list = df_etl_files_list[df_etl_files_list['extension'] == 'json']

# iterate through the list and export to csv files
for i in range(len(df_json_list)):
    in_json = df_json_list['full_path'].iloc[i]
    out_csv = path_etl + "cleansing\\" + in_json.rsplit("\\")[-1].replace('.json', '_csv.csv')

    with open(in_json, 'r') as f:
        json_file = json.load(f)

    export_json_to_csv(json_file, out_csv)


JSON data was exported to "c:\Users\Diane\git\data_analysis\Nestle\etl\cleansing\Vendas_formatted_csv.csv"


In [13]:
# This function returns a data frame object based on listed file
def get_raw_file(full_path):
    if full_path.split('.')[1] == 'csv':
        df_raw = pd.read_csv(full_path, delimiter=",")
    elif full_path.split('.')[1] == 'xlsx':
        df_raw = pd.read_excel(full_path)
    elif full_path.split('.')[1] == 'json':
        df_raw = pd.read_json(full_path)

    return df_raw

In [14]:
# List all files in etl directory
df_etl_files_list = list_files(path_etl, 1)
df_list = df_etl_files_list[df_etl_files_list['extension'] == 'csv']
# display(df_list)

df__list_vendas = df_list.loc[df_list['file'].str.contains('Vendas')].values[0]
f_name = df__list_vendas[0]

df_vendas = get_raw_file(f_name)

In [15]:
# select head df
df_head = df_vendas.columns.tolist()

# select columns with client's info
df_head_client = df_head[:15]

i = 1
while i <= 12:
    # select columns that contains 'meses'
    df_head_mes = [j for j in df_head if ('meses.'+str(i))+'.' in j]
    
    df_head_to_filter = df_head_client + df_head_mes

    df_meses = df_vendas[df_head_to_filter]
    
    out_csv = path_etl + 'cleansing\\' + 'vendas_mes_' + str(i) + '_formatted.csv'
    df_meses.to_csv(out_csv, index=False)

    i += 1
